In [1]:
import os
import torch
import gc
import logging

gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()        
    torch.cuda.ipc_collect()


try:
    %run setup_paths
except:
    %run notebooks/setup_paths
    

logging.basicConfig(
    level=logging.INFO,  # or DEBUG, WARNING, etc.
    format='%(asctime)s - %(levelname)s - %(message)s',
    stream=sys.stdout
)

logging.info(f"current dir: {os.getcwd()}")

2025-08-19 00:22:14,912 - INFO - current dir: c:\Projects\scmsim


## Fetch and observe the data
- After it was generated CausalSim.MovieLens1M.0.MF40 was added to configs/config.yaml
- The .0 - is because it was trained on the sample-set 0

In [11]:
%run src/utils
%run src/loaders

cfg = read_cfg('configs/config.yaml')
paths = PathProvider(cfg['paths']['results'], cfg['paths']['products'])
uidata = MovieLensData(get_uidata_loader(cfg, 'CausalSim.MovieLens1M.0.MF40'))
uidata.ratings

2025-08-19 06:50:30,180 - INFO - loading ratings


,user_id,item_id,watched,timestamp
0,1,1,1.0,0.158135
1,1,34,1.0,0.953114
2,1,57,1.0,1.175028
3,1,268,1.0,1.789184
4,1,296,1.0,0.328906
...,...,...,...,...
1050442,6040,3819,1.0,0.142307
1050443,6040,3858,1.0,0.821029
1050444,6040,3868,1.0,0.094026
1050445,6040,3875,1.0,0.474089


## Generate estimations on top of trained models
- IPWPaeams(clipping, stabalized)
- (the last setting is stabalized with clipping=1.0 - which sets a weight of 1 to each sample.)

In [22]:

%run src/estimators
%run src/pipeline
%run src/estimators

ground_truth_name = 'CausalSim.MovieLens1M.MF40/gt.filtered.8'
ipw_params = [IPWParams(0, False), IPWParams(0, True), IPWParams(0.2, True), IPWParams(1, True)]

for model_name in ['MF30', 'MF50', 'MF60']:
    model = load_model(paths, uidata.name(), model_name)
    ipwe = MFIPWEstimator(model_name, model, ipw_params)    
    ## reset_ids mean we re-match the item-ids from the title name 
    create_estimations(paths, uidata, f'{model_name}.IPW', ground_truth_name, ipwe, reset_ids=False)



2025-08-19 06:55:59,230 - INFO - loading model MF30 (version 0) at results\CausalSim.MovieLens1M.0.MF40\models\MF30.0\weights.pt
2025-08-19 06:55:59,292 - INFO - processing: 0:4096 / 6212


C:\Projects\scmsim\src\loaders.py:56: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(model_path)


2025-08-19 06:56:02,985 - INFO - processing: 4096:6212 / 6212
2025-08-19 06:56:05,055 - INFO - saving estimations at results\CausalSim.MovieLens1M.0.MF40\estimations\CausalSim.MovieLens1M.MF40.gt.filtered.8\MF30.IPW
2025-08-19 06:56:05,268 - INFO - loading model MF50 (version 0) at results\CausalSim.MovieLens1M.0.MF40\models\MF50.0\weights.pt
2025-08-19 06:56:05,412 - INFO - processing: 0:4096 / 6212


C:\Projects\scmsim\src\loaders.py:56: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(model_path)


2025-08-19 06:56:08,518 - INFO - processing: 4096:6212 / 6212
2025-08-19 06:56:10,424 - INFO - saving estimations at results\CausalSim.MovieLens1M.0.MF40\estimations\CausalSim.MovieLens1M.MF40.gt.filtered.8\MF50.IPW
2025-08-19 06:56:10,642 - INFO - loading model MF60 (version 0) at results\CausalSim.MovieLens1M.0.MF40\models\MF60.0\weights.pt
2025-08-19 06:56:10,790 - INFO - processing: 0:4096 / 6212


C:\Projects\scmsim\src\loaders.py:56: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(model_path)


2025-08-19 06:56:14,167 - INFO - processing: 4096:6212 / 6212
2025-08-19 06:56:16,263 - INFO - saving estimations at results\CausalSim.MovieLens1M.0.MF40\estimations\CausalSim.MovieLens1M.MF40.gt.filtered.8\MF60.IPW


## Generate basic ertimations

In [23]:
basee = BasicEstimator()
create_estimations(paths, uidata, 'Basic' , ground_truth_name, basee)


2025-08-19 06:56:16,633 - INFO - processing: 0:4096 / 6212
2025-08-19 06:56:18,352 - INFO - processing: 4096:6212 / 6212
2025-08-19 06:56:19,242 - INFO - saving estimations at results\CausalSim.MovieLens1M.0.MF40\estimations\CausalSim.MovieLens1M.MF40.gt.filtered.8\Basic


,CORR,LIFT,SATE,treatment_idx,resp_idx
0,-0.021775,2.415570,0.049250,1254,1515
1,0.380641,3.062363,0.066917,166,1515
2,-0.068986,1.330297,0.164616,1254,648
3,0.431802,1.382588,0.285350,1197,648
4,0.043527,1.952166,0.472681,376,648
...,...,...,...,...,...
6207,-0.149513,1.780210,0.337812,707,1198
6208,-0.216942,2.185692,0.054795,1213,1753
6209,0.080965,1.985557,0.163326,1114,292
6210,-0.053662,1.746397,0.071957,695,7


## Method Comparison
- MF30.IPW - is regular IPW on top of MF30 model
- MF30.IPW.s - is stabilized IPW on top of MF30 model
- MF60.IPWs.clp0.2.s is stabilized IPW with clipping=0.2 on top of MF60 model

In [24]:
get_sim_scores(paths, uidata, ground_truth_name)

,name,mse,mse_pos,mse_zero
0,CORR,0.025124499956625156,0.021738866091250895,0.025491851298430734
1,LIFT,10.62419168894716,52.018035459608654,7.250225137645611
2,SATE,0.0372008180172848,0.05154356699344777,0.03432725749338219
3,MF30.IPW,0.02371982113007648,0.07001727522728339,0.0198038417871699
4,MF30.IPW.s,0.001976229726687486,0.035347683771008664,3.1103954317938485e-05
5,MF30.IPW.clp0.2.s,0.003282956492079736,0.04036590701237086,0.0008955471175844678
6,MF30.IPW.clp1.s,0.02371982113007648,0.07001727522728339,0.0198038417871699
7,MF50.IPW,0.02371982113007648,0.07001727522728339,0.0198038417871699
8,MF50.IPW.s,0.0019690970201697968,0.03534150604566404,2.716205201322971e-05
9,MF50.IPW.clp0.2.s,0.00329131804132819,0.04039089309609428,0.0009019903068463119
